## Importing Libraries

In [10]:
from pprint import pprint
import pandas as pd
import boto3
import os
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
sns.set_style("darkgrid")

### Reading the data

In [2]:
df = pd.read_json(r"E:\customer_churn_prediction\data\processed\df_cleaned_events.json", lines= True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278154 entries, 0 to 278153
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ts             278154 non-null  int64  
 1   userId         278154 non-null  int64  
 2   sessionId      278154 non-null  int64  
 3   page           278154 non-null  object 
 4   auth           278154 non-null  object 
 5   method         278154 non-null  object 
 6   status         278154 non-null  int64  
 7   level          278154 non-null  object 
 8   itemInSession  278154 non-null  int64  
 9   location       278154 non-null  object 
 10  userAgent      278154 non-null  object 
 11  lastName       278154 non-null  object 
 12  firstName      278154 non-null  object 
 13  registration   278154 non-null  int64  
 14  gender         278154 non-null  object 
 15  artist         228108 non-null  object 
 16  song           228108 non-null  object 
 17  length         228108 non-nul

Converting ts and registration to datetime

In [4]:
df['ts'] = pd.to_datetime(df['ts'], unit='ms')
df['registration'] = pd.to_datetime(df['registration'], unit='ms')

In [9]:
df = df.sort_values(["userId", "ts"])
df["next_action"] = df.groupby("userId")["page"].shift(-1)
events_post_cancellation = df[
    (df["page"] == "Cancellation Confirmation") & (df["next_action"].notna())
]
print("Events after cancellation:", len(events_post_cancellation))


Events after cancellation: 0


In [ ]:
#Every user gets churn = 1 if they ever hit "Cancellation Confirmation", else 0.
user_churn = (
    df.groupby("userId")["page"]
      .apply(lambda x: np.where(x.eq("Cancellation Confirmation").any(), 1, 0))
      .reset_index(name="churn")
)

In [12]:
df = df.merge(user_churn, on="userId", how="left")

In [14]:
df.churn.value_counts()

churn
0    233290
1     44864
Name: count, dtype: int64